In [11]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [12]:
# Function to check and install specific package versions
def install_package(package, version):
    try:
        # Check the current version
        current_version = __import__(package).__version__

        # Special handling for torch to check for version with CUDA suffix
        if package == "torch":
            if current_version.startswith(version):
                print(f"{package} version {version} is already installed.")
                return
        else:
            if current_version == version:
                print(f"{package} version {version} is already installed.")
                return

        print(f"Uninstalling {package} version {current_version}...")
        !pip uninstall -y {package}
        print(f"Installing {package} version {version}...")
        !pip install {package}=={version}

    except ImportError:
        print(f"{package} not installed. Installing {package} version {version}...")
        !pip install {package}=={version}

# Specify the packages and their desired versions
packages = {
    "pandas": "2.2.2",
    "numpy": "1.26.4",
    "scikit-learn": "1.6.0",
    "torch": "2.5.1",
    "joblib": "1.4.2",
    "matplotlib": "3.10.0"
}

# Check and install packages
for pkg, ver in packages.items():
    install_package(pkg, ver)

# Import the necessary libraries after ensuring correct versions
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset, Subset
from sklearn.model_selection import KFold
import joblib
import matplotlib as mpl  # Import matplotlib for version checking
import matplotlib.pyplot as plt
from datetime import datetime, time
import random
from sklearn import __version__ as sklearn_version  # Import sklearn version
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import ExponentialLR

# Print package versions
print("pandas version:", pd.__version__)
print("numpy version:", np.__version__)
print("scikit-learn version:", sklearn_version)
print("torch version:", torch.__version__)
print("joblib version:", joblib.__version__)
print("matplotlib version:", mpl.__version__)
print("CUDA available:", torch.cuda.is_available())

pandas version 2.2.2 is already installed.
numpy version 1.26.4 is already installed.
scikit-learn not installed. Installing scikit-learn version 1.6.0...
torch version 2.5.1 is already installed.
joblib version 1.4.2 is already installed.
matplotlib version 3.10.0 is already installed.
pandas version: 2.2.2
numpy version: 1.26.4
scikit-learn version: 1.6.0
torch version: 2.5.1+cu124
joblib version: 1.4.2
matplotlib version: 3.10.0
CUDA available: True


In [13]:
# DETREND_WINDOW_SIZE = 2
BATCH_SIZE = 2048
WINDOW_SIZE = 50
LEARNING_RATE = 0.001  # From 0.001 to the best 0.0001
EPOCH_SIZE = 3 # 50
PREDICTION_LEN = 1   # Number of time steps to predict each time
K_FOLDS = 3 # 10  # Number of folds for cross-validation
TIME_INTERVAL_SEC = 60  # For 1 minute interval: 60
PREDICTION_TIMESTEP_SPAN = 0  # 14

In [14]:
# Load the JSON data
# data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/futures.ai/spy_1min_regularhours_truncated_preprocessed.json')
data = pd.read_json('/content/drive/MyDrive/Colab Notebooks/futures.ai/spy_1min_regularhours_truncated_semisupervised_preprocessed.json')

# Convert 'datetime' strings back to datetime objects
data['datetime'] = pd.to_datetime(data['datetime'])

## De-trend the data
#
# De-trending helps to remove any long-term trends or seasonality from the data,
# allowing the model to focus on the fluctuations that matter most for prediction.
#
# Why De-trend Before Normalization?
# Focus on Fluctuations:
# Normalizing data with trends can make it difficult for the model to learn meaningful patterns,
# as the model may focus on the trends rather than the actual relationships between features.
# Consistent Scale:
# By de-trending all relevant features, you ensure that they are on a similar scale, which can improve the learning process.
FEATURES_TO_DETREND = ['close', 'EMA_5', 'EMA_10', 'EMA_15', 'EMA_20']  #'open', 'high', 'low',
FEATURES_NOT_TO_DETREND = ['LONG_UPPER_SHADOW', 'LONG_LOWER_SHADOW', 'EMA_5_EMA_10', 'EMA_15_EMA_20', 'RSI', 'RSI_INT']
# FEATURES_NOT_TO_DETREND = ['RSI']
ALL_FEATURES = FEATURES_TO_DETREND + FEATURES_NOT_TO_DETREND

for feature in FEATURES_TO_DETREND:
    # SMA for de-trending
    #
    # Choosing the Window Size for Rolling Mean
    #
    # If your data has short-term fluctuations, a smaller DETREND_WINDOW_SIZE (e.g., 5-10)
    # might be more appropriate, allowing you to capture more immediate trends.
    # For longer-term trends, a larger DETREND_WINDOW_SIZE (e.g., 30-60) is often used
    # to smooth out more significant fluctuations and focus on overarching trends.
    #
    # Consider what you are trying to predict. If you are interested in short-term price movements,
    # a smaller window might be better. For long-term predictions, a larger window can help smooth out noise.
    #
    # If your goal is to analyze short-term trends without losing the fluctuations, you might consider using a very short window size (e.g., 2 or 3)
    # for a moving average. This would still smooth the data slightly but would retain more of the original fluctuations compared to larger windows.
    # data[f'{feature}_trend'] = data[feature].rolling(window=DETREND_WINDOW_SIZE).mean()  # Simple moving average
    # data[f'{feature}_detrended'] = data[feature] - data[f'{feature}_trend']

    # Differencing for de-trending
    data[f'{feature}_detrended'] = data[feature].diff()  # Calculate the difference from the previous value
data = data.dropna()    # Drop the first row which will be NaN due to differencing or drop NaN values created by rolling mean

## Normalize the de-trended features

# Define fixed scaler for RSI
class FixedRangeScaler:
    def __init__(self, feature_range=(0, 1), input_range=(0, 100)):
        self.feature_range = feature_range
        self.input_range = input_range

    def transform(self, X):
        X_std = (X - self.input_range[0]) / (self.input_range[1] - self.input_range[0])
        X_scaled = X_std * (self.feature_range[1] - self.feature_range[0]) + self.feature_range[0]
        return X_scaled

    def inverse_transform(self, X):
        X_std = (X - self.feature_range[0]) / (self.feature_range[1] - self.feature_range[0])
        X_original = X_std * (self.input_range[1] - self.input_range[0]) + self.input_range[0]
        return X_original

# Initialize scalers
dynamic_scaler = MinMaxScaler(feature_range=(0, 1))
long_upper_shadow_dynamic_scaler = MinMaxScaler(feature_range=(0, 1))
long_lower_shadow_dynamic_scaler = MinMaxScaler(feature_range=(0, 1))
rsi_scaler = FixedRangeScaler(feature_range=(0, 1), input_range=(0, 100))
rsi_int_scaler = FixedRangeScaler(feature_range=(0, 1), input_range=(-1, 1))

data = data.copy()  # Create a copy to avoid SettingWithCopyWarning
data[[f'{feature}_normalized' for feature in FEATURES_TO_DETREND]] = dynamic_scaler.fit_transform(
    data[[f'{feature}_detrended' for feature in FEATURES_TO_DETREND]].values
)
data['LONG_UPPER_SHADOW_normalized'] = long_upper_shadow_dynamic_scaler.fit_transform(data['LONG_UPPER_SHADOW'].values.reshape(-1, 1))
data['LONG_LOWER_SHADOW_normalized'] = long_lower_shadow_dynamic_scaler.fit_transform(data['LONG_LOWER_SHADOW'].values.reshape(-1, 1))
data['EMA_5_EMA_10_normalized'] = data['EMA_5_EMA_10']
data['EMA_15_EMA_20_normalized'] = data['EMA_15_EMA_20']
data['RSI_normalized'] = rsi_scaler.transform(data['RSI'].values.reshape(-1, 1))
data['RSI_INT_normalized'] = rsi_int_scaler.transform(data['RSI_INT'].values.reshape(-1, 1))

# Save the normalized and de-trended data for future use
# data.to_json('processed_data.json', orient='records')

# Save the scalers
joblib.dump(dynamic_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_dynamic_scaler.pkl')
joblib.dump(long_upper_shadow_dynamic_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_long_upper_shadow_dynamic_scaler.pkl')
joblib.dump(long_lower_shadow_dynamic_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_long_lower_shadow_dynamic_scaler.pkl')
joblib.dump(rsi_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_rsi_scaler.pkl')
joblib.dump(rsi_int_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_rsi_int_scaler.pkl')

# Fit a dedicated scaler just for 'close'
close_dynamic_scaler = MinMaxScaler(feature_range=(0, 1))
data['close_normalized'] = close_dynamic_scaler.fit_transform(data[['close_detrended']].values)

# Save the close scaler
joblib.dump(close_dynamic_scaler, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_close_dynamic_scaler.pkl')

['/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_close_dynamic_scaler.pkl']

In [15]:
## Define a PyTorch Dataset
# Modified TimeSeriesDataset with strict day boundary checks for day-after-day training
class TimeSeriesDataset(Dataset):
    def __init__(self, data):
        self.sequence_length = WINDOW_SIZE
        self.prediction_length = PREDICTION_LEN

        # Convert the 'datetime' strings to numpy datetime64 for efficient processing
        self.dates = np.array([np.datetime64(dt) for dt in data['datetime']])

        # Extract features as numpy arrays
        self.features = np.stack([
            data[f'{feature}_normalized'].values
            for feature in ALL_FEATURES
        ], axis=1)  # Shape: (n_samples, n_features)

        # Store indices for each day
        unique_dates = np.unique(self.dates.astype('datetime64[D]'))
        print(f'unique_dates: {unique_dates}')
        print(f'unique_dates length: {len(unique_dates)}')
        self.daily_indices = {
            date: np.where(self.dates.astype('datetime64[D]') == date)[0] for date in unique_dates
        }

        # Pre-compute valid indices for each day
        self.valid_indices = []
        for date, day_indices in self.daily_indices.items():
            if len(day_indices) == 0:
                continue  # Skip if there are no indices for this date

            for i in range(len(day_indices)):
                seq_start = i
                seq_end = i + self.sequence_length
                target_start = seq_end + PREDICTION_TIMESTEP_SPAN
                target_end = target_start + self.prediction_length

                # Check if all indices are within the same day
                if target_end <= len(day_indices):
                    # Verify time continuity
                    if self._verify_time_continuity(day_indices, seq_start, seq_end, target_start, target_end):
                        self.valid_indices.append((
                            day_indices[seq_start:seq_end],
                            day_indices[target_start:target_end]
                        ))
        print(f'self.valid_indices head: {self.valid_indices[:10]}')
        print(f'self.valid_indices tail: {self.valid_indices[-10:]}')
        print(f'self.valid_indices length: {len(self.valid_indices)}')

    def _verify_time_continuity(self, day_indices, seq_start, seq_end, target_start, target_end):
        # Verify that times are continuous
        for j in range(seq_start, seq_end - 1):
            t1 = self.dates[day_indices[j]]
            t2 = self.dates[day_indices[j + 1]]
            if (t2 - t1).astype('timedelta64[s]') != TIME_INTERVAL_SEC:
                return False

        for j in range(target_start, target_end - 1):
            t1 = self.dates[day_indices[j]]
            t2 = self.dates[day_indices[j + 1]]
            if (t2 - t1).astype('timedelta64[s]') != TIME_INTERVAL_SEC:
                return False

        return True

    def __len__(self):
        # Total number of items across all days
        return len(self.valid_indices)

    def __getitem__(self, index):
        seq_indices, target_indices = self.valid_indices[index]

        # Return data in (features, sequence) format

        # Get input sequence
        x = self.features[seq_indices]  # Shape: (sequence_length, n_features)
        x = torch.tensor(x, dtype=torch.float32)

        # Get target sequence
        # Change in target extraction
        # y = self.features[target_indices]   # Target is now all normalized features for the next timestep(s)
        # y = torch.tensor(y, dtype=torch.float32)  # Shape: (prediction_length, n_features)
        y = self.features[target_indices][:, 0]  # Only extracting the 'close' price
        y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Shape: (prediction_length, 1)

        return x, y

In [16]:
class GRU(nn.Module):
    def __init__(self):
        super(GRU, self).__init__()
        self.input_size = len(ALL_FEATURES)
        # Why self.input_size = WINDOW_SIZE * len(ALL_FEATURES) and not self.input_size = len(ALL_FEATURES)?
        # In the ANN model, the input is flattened into a single vector because ANNs (fully connected networks)
        # do not inherently handle sequential data. The input shape is transformed from (batch_size, sequence_length, n_features)
        # to (batch_size, sequence_length * n_features) using nn.Flatten(). This means the entire sequence is treated as a single input vector.
        # However, in the GRU model, the input is processed sequentially,
        # and the GRU layer expects the input to have the shape (batch_size, sequence_length, n_features). Here:
        # - sequence_length is the number of time steps in the sequence (e.g., WINDOW_SIZE).
        # - n_features is the number of features at each time step (e.g., len(ALL_FEATURES)).
        # So, the GRU model does not flatten the input. Instead, it processes the sequence step by step,
        # maintaining the temporal structure of the data. Therefore:
        # - input_size in the GRU model refers to the number of features at each time step (len(ALL_FEATURES)),
        # not the total size of the flattened input (WINDOW_SIZE * len(ALL_FEATURES)).
        self.hidden_size = 256
        self.num_layers = 2
        self.output_size = PREDICTION_LEN  # Output Layer with only the 'close' price

        # GRU Layer
        self.gru = nn.GRU(
            self.input_size,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            # dropout=0.2 if self.num_layers > 1 else 0
            )

        # Fully connected layer (Dense) to map GRU output to the desired output size
        # self.fc = nn.Linear(self.hidden_size, self.output_size)

        # Additional Dense layers
        self.fc1 = nn.Linear(self.hidden_size, 128)  # First Dense layer
        self.fc2 = nn.Linear(128, 64)  # Second Dense layer
        self.fc3 = nn.Linear(64, self.output_size)  # Final output layer

        # Activation functions
        self.relu = nn.ReLU() # With Dense layers
        self.sigmoid = nn.Sigmoid() # Optional: You can add a sigmoid or softmax layer if needed

    def forward(self, x):
        # Input shape: (batch_size, sequence_length, n_features)
        batch_size = x.size(0)
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)

        # Forward propagate GRU
        out, _ = self.gru(x, h0)  # Shape: (batch_size, sequence_length, hidden_size)
        # Use the last PREDICTION_LEN time steps
        out = out[:, -PREDICTION_LEN:, :]  # Shape: (batch_size, PREDICTION_LEN, hidden_size)

        # Apply fully connected layer to each time step
        # out = self.fc(out)  # Shape: (batch_size, PREDICTION_LEN, output_size)

        # Apply Dense layers
        out = self.relu(self.fc1(out))  # Shape: (batch_size, PREDICTION_LEN, 128)
        out = self.relu(self.fc2(out))  # Shape: (batch_size, PREDICTION_LEN, 64)
        out = self.fc3(out)  # Shape: (batch_size, PREDICTION_LEN, output_size)

        out = self.sigmoid(out) # Apply sigmoid if needed
        return out

In [17]:
## Train the model
def train_with_validation(model, train_loader, val_loader, criterion, optimizer, epochs=EPOCH_SIZE):
    # Modified training with model state saving
    best_val_loss = float('inf')
    best_fold_state = None
    patience = 5
    patience_counter = 0
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Learning rate scheduler
    # scheduler = StepLR(optimizer, step_size=10, gamma=0.1)
    # scheduler = ExponentialLR(optimizer, gamma=0.9)

    # Adaptive Learning Rate Adjustment
    # prev_val_loss = float('inf')
    # min_lr = 1e-6  # Minimum learning rate
    # max_lr = 1e-2  # Maximum learning rate

    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')

        # Training phase
        model.train()
        train_loss = 0
        for x_batch, y_batch in train_loader:
            x_batch = x_batch.to(device, non_blocking=True)  # non_blocking=True for async transfer
            y_batch = y_batch.to(device, non_blocking=True)

            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)

            # L1 regularization
            # l1_lambda = 1e-5
            # l1_norm = sum(p.abs().sum() for p in model.parameters())
            # loss = loss + l1_lambda * l1_norm

            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch = x_batch.to(device, non_blocking=True)  # non_blocking=True for async transfer
                y_batch = y_batch.to(device, non_blocking=True)
                outputs = model(x_batch)
                val_loss += criterion(outputs, y_batch).item()

        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)

        print(f'Train Loss: {avg_train_loss:.8f}, Validation Loss: {avg_val_loss:.8f}')

        # Adaptive Learning Rate Adjustment
        # new_learning_rate = 0
        # if avg_val_loss < prev_val_loss:
        #     # Increase learning rate by 10%
        #     for param_group in optimizer.param_groups:
        #         param_group['lr'] = min(param_group['lr'] * 1.1, max_lr)  # Ensure it doesn't exceed max_lr
        #         new_learning_rate = param_group['lr']
        #     print(f'Learning Rate: {new_learning_rate}')
        # else:
        #     # Decrease learning rate by 50%
        #     for param_group in optimizer.param_groups:
        #         param_group['lr'] = max(param_group['lr'] * 0.5, min_lr)  # Ensure it doesn't go below min_lr
        #         new_learning_rate = param_group['lr']
        #     print(f'Learning Rate: {new_learning_rate}')
        # prev_val_loss = avg_val_loss

        # Step the learning rate scheduler
        # scheduler.step()
        # print(f'Learning Rate: {scheduler.get_last_lr()[0]}')

        # Early stopping logic
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_fold_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                break

    return best_val_loss, best_fold_state

In [18]:
def split_dates_into_folds():
    # Get unique dates and sort them
    unique_dates = sorted(set(pd.to_datetime(data['datetime']).dt.date))
    total_dates = len(unique_dates)

    # Calculate size of each fold
    fold_size = total_dates // K_FOLDS

    # Create non-overlapping folds
    folds = []
    for i in range(K_FOLDS):
        start_idx = i * fold_size
        end_idx = start_idx + fold_size if i < K_FOLDS - 1 else total_dates
        fold_dates = unique_dates[start_idx:end_idx]
        folds.append(fold_dates)

    return folds

In [19]:
def cross_validate():
    full_dataset = TimeSeriesDataset(data)
    folds = split_dates_into_folds()

    best_overall_loss = float('inf')
    best_overall_state = None
    all_fold_losses = []  # To store losses for each fold

    for fold_idx in range(K_FOLDS):
        # Use current fold as validation, all others as training
        val_dates = set(folds[fold_idx])
        # Shuffle the fold indices except the current fold_idx
        fold_indices = list(range(K_FOLDS))
        fold_indices.remove(fold_idx)
        random.shuffle(fold_indices)
        train_dates = set()
        for i in fold_indices:
            train_dates.update(folds[i])

        print(f'Fold {fold_idx + 1}:')
        print(f'Training dates: from {min(train_dates)} to {max(train_dates)}')
        print(f'Validation dates: from {min(val_dates)} to {max(val_dates)}')

        # Split indices
        train_indices = []
        val_indices = []
        for idx, (seq_indices, target_indices) in enumerate(full_dataset.valid_indices):
            date = pd.to_datetime(data.iloc[seq_indices[0]]['datetime']).date()
            if date in train_dates:
                train_indices.append(idx)
            elif date in val_dates:
                val_indices.append(idx)

        print(f'Training samples: {len(train_indices)}, Validation samples: {len(val_indices)}')

        train_dataset = Subset(full_dataset, train_indices)
        val_dataset = Subset(full_dataset, val_indices)

        # Create data loaders for train and validation splits
        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # Initialize a new model, criterion, and optimizer for each fold
        model = GRU()
        criterion = nn.MSELoss()
        # criterion = nn.SmoothL1Loss()  # Huber loss
        # criterion = nn.L1Loss()
        # def msle_loss(outputs, targets):
        #     return torch.mean((torch.log(outputs + 1) - torch.log(targets + 1)) ** 2)
        # criterion = msle_loss
        # def custom_loss(outputs, targets):
        #     mse_loss = nn.MSELoss()(outputs, targets)
        #     mae_loss = nn.L1Loss()(outputs, targets)
        #     return 0.7 * mse_loss + 0.3 * mae_loss
        # criterion = custom_loss
        optimizer = torch.optim.Adam(  # Changed from Adam to AdamW for weight_decay (L2)
            model.parameters(),
            lr=LEARNING_RATE,
            # weight_decay=1e-5   # If needed, increase the weight_decay (L2) from 1e-5 to 1e-4
        )

        # Train and validate the model
        best_fold_loss, best_fold_state = train_with_validation(
            model,
            train_loader,
            val_loader,
            criterion,
            optimizer
        )

        all_fold_losses.append(best_fold_loss)  # Store loss for this fold
        print(f'Validation Loss for fold {fold_idx+1}: {best_fold_loss:.8f}')

        # Save the best model
        if best_fold_loss < best_overall_loss:
            best_overall_loss = best_fold_loss
            best_overall_state = best_fold_state

    # Print the results
    print('\nCross-validation complete!')
    print(f'Average validation loss: {np.mean(all_fold_losses):.8f}')
    print(f'Standard deviation: {np.std(all_fold_losses):.8f}')

    # Save the best model
    torch.save(best_overall_state, '/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_best_model_cv.pth')

    # Plotting the validation losses
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, K_FOLDS + 1), all_fold_losses, marker='o', linestyle='-')
    plt.title('Validation Loss Across Folds')
    plt.xlabel('Fold Number')
    plt.ylabel('Validation Loss')
    plt.xticks(range(1, K_FOLDS + 1))
    plt.grid()
    plt.savefig('/content/drive/MyDrive/Colab Notebooks/futures.ai/gru_cross_validation_loss.png')  # Save the plot as an image
    plt.close()  # Close the plot to free up memory

    return best_overall_state, best_overall_loss

In [20]:
# Perform cross-validation
best_overall_state, best_overall_loss = cross_validate()

# Create a final model with the best weights
# final_model = ANN()
# final_model.load_state_dict(best_overall_state)

print(f'\nBest model saved with validation loss: {best_overall_loss:.8f}')

unique_dates: ['2016-05-26' '2016-05-27' '2016-05-31' ... '2024-05-24' '2024-05-28'
 '2024-05-29']
unique_dates length: 2015
self.valid_indices head: [(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]), array([50])), (array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]), array([51])), (array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]), array([52])), (array([ 3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
       20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 